In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Training Model

In [2]:
import pickle
import spacy
import random
from spacy.util import minibatch, compounding
from spacy import load, displacy

In [3]:
# load datasets 

with open('/content/drive/MyDrive/CSIT_Internship/ner_spacy_fmt_datasets.pickle', 'rb') as f:
    ner_spacy_fmt_datasets = pickle.load(f)

In [4]:
nlp=spacy.blank("id")

In [5]:
nlp.add_pipe('ner')

In [6]:
nlp.begin_training()

In [7]:
import random
from spacy.util import minibatch, compounding

In [8]:
ner=nlp.get_pipe("ner")

In [9]:
for _, annotations in ner_spacy_fmt_datasets:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])
        break

In [10]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
# TRAINING MODEL
from spacy.training import Example
examples = []
for text, annots in ner_spacy_fmt_datasets:
    examples.append(Example.from_dict(nlp.make_doc(text), annots))
nlp.initialize(lambda: examples)
for i in range(12):
    random.shuffle(examples)
    for batch in minibatch(examples, size=8):
        nlp.update(batch)

In [ ]:
# save model 
from pathlib import Path

output_dir = Path('/content/drive/MyDrive/CSIT_Internship/nlp_id_checkpoint_2022_09_22_03')
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

## Evaluating Model

In [ ]:
# load existing model 
output_dir = '/content/drive/MyDrive/CSIT_Internship/nlp_id_checkpoint_2022_09_22_03'
print("Loading from", output_dir)
nlp_updated = spacy.load(output_dir)

In [ ]:
f = open("/content/drive/MyDrive/CSIT_Internship/data/indotest.txt")
toPredict = ''
gold_words = []
gold_ents = []
for line in f:
  if line!="\n":
    #print(line)
    word, tag = line.split()
    #print(word)
    gold_words.append(word)
    newtag=tag[2:]
    if newtag=="PLACE":
      newtag="LOCATION"
      newtag=tag[0:2]+newtag
      gold_ents.append(newtag)
    elif newtag=="ORGANISATION":
      newtag="ORGANIZATION"
      newtag=tag[0:2]+newtag
      gold_ents.append(newtag)
    else:
      gold_ents.append(tag)
    toPredict=toPredict+" "+word
f.close()
toPredict=toPredict[1:]
print((gold_ents))
print(len(gold_ents))

In [ ]:
# test 
doc = nlp_updated(toPredict)
first_ents=[]
first_words=[]
pred_words=[]
pred_ents=[]
#print(doc.ents)
print(len(toPredict.split()))
num=0
for ent in doc.ents:
  for i in range(len(ent.text.split())):
    if i==0:
      first_ents.append("B-"+ent.label_)
      first_words.append(ent.text.split()[i])
    else:
      first_ents.append("I-"+ent.label_)
      first_words.append(ent.text.split()[i])
wordsInText=toPredict.split()
for word in wordsInText:
  if num<len(first_words):
    if word!=first_words[num]:
      pred_words.append(word)
      pred_ents.append("O")
    else:
      pred_words.append(first_words[num])
      if first_ents[num]=="B-TIME" or first_ents[num]=="I-TIME" or first_ents[num]=="B-QUANTITY" or first_ents[num]=="I-QUANTITY":
        pred_ents.append("O")
      else:
        pred_ents.append(first_ents[num])
      num+=1
  else:
    pred_words.append(word)
    pred_ents.append("O")
      

    
print(pred_words)
print(len(pred_words))
print(pred_ents)
print(len(pred_ents))
#print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
from spacy.tokens import Doc
from spacy.training import Example
predicted = Doc(nlp_updated.vocab, words=pred_words, ents=pred_ents)
reference = Doc(nlp_updated.vocab, words=gold_words, ents=gold_ents)
example = Example(predicted, reference)

In [ ]:
from spacy.scorer import Scorer

scorer = Scorer()
scores = scorer.score([example])
print(scores)

# Testing Model by loading PDFs

In [18]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 218 kB 26.9 MB/s 


In [19]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk import stem
import os
import PyPDF2
from PyPDF2 import PdfFileReader
import numpy as np
from sklearn.feature_extraction import text
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [20]:
def pdf2text(pdf):
    '''Iterate over pages and extract text'''
    text = ''
    for i in range(pdf.getNumPages()):
        page = pdf.getPage(i)
        text = text + page.extractText()
    return text
 
def stem_tokenize(document):
    '''return stemmed words longer than 2 chars and all alpha'''
    tokens = [stem(w) for w in document.split() if len(w) > 2 and w.isalpha()]
    return tokens
 
def tokenize(document):
    '''return words longer than 2 chars and all alpha'''
    tokens = [w for w in document.split() if len(w) > 2 and w.isalpha()]
    return tokens
 
def build_corpus_from_dir(dir_path):
    corpus = []
    for root, dirs, filenames in os.walk(dir_path):
        for name in filenames:
            print(name)
            f  = os.path.join(root, name)
            pdf = PdfFileReader(f, 'rb')
            document = pdf2text(pdf)
            corpus.append(document)
    return corpus

In [26]:
corpus = build_corpus_from_dir('/content/drive/MyDrive/CSIT_Internship/indo_news')

indo_news1.pdf


In [27]:
test_text =''
for i in corpus:
  test_text=test_text+i
print(test_text)

Inggris dan dunia mengucapkan salam perpisahan yang terakhir kalinya kepada Ratu Elizabeth II di 
pemakaman kenegaraan, yang digelar Senin kemarin (19/09).  
 
Dihadiri para pemimpin dunia dan tentunya ratusan ribu warga di jalanan kota London, semua mata 
tertuju pada peti yang membawa jenazah mendiang Ratu Elizabeth II.  
 
Sebagai negara yang identik dengan kemewahan, pemakaman Ratu menjadi yang pemakaman 
kenegara an pertama kalinya dilakukan sejak pemakaman mantan perdana menteri Inggris Winston 
Churchill.  
 
Sebelum prosesi pemakaman, bel dibunyikan sebanyak 96 kali dalam setiap menit, menandakan usia 
Ratu Elizabeth II.  
 
Sebanyak 142 pelaut Angkatan Laut Kerajaan me narik kereta yang mengangkut peti mati ratu yang 
diselimuti bendera menuju Westminister Abbey.  
 
Di Westminister Abbey, 2.000 orang dari seluruh dunia, mulai dari pemimpin dunia hingga tenaga 
kesehatan berkumpul untuk menyaksikan upacara pemakaman.  
 
kartu d ari Raja Charles III yang ditulis tangannya send

In [28]:
doc = nlp_updated(test_text)
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)